In [12]:
from typing import List
from typing import Tuple
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

# 1.
A neural network is a machine learning model that tries to mimic a brain by "thinking". It assigns weights to variables and makes changes to the weights by running the model multiple times and getting error feedback. A NN has 3 main parts: Input Layer, Output Layer, and hidden layers where the thinking occurs. Building a NN requires 2 main things, the function that changes the variables at each node to do the thinking (activation function) and the loss function that tells the machine by how much its prediction was off.

# 2.
A loss function is the simplest way to check the performance of a NN because it shows the prediction error. Another good way to check is to use k-fold cross validation, because it lets every part of the data be tested at least once and runs multiple tests instead of just one.

In [13]:
df = pd.read_csv('Credit_Card.csv')
label_df = pd.read_csv('Credit_card_label.csv')
sorted_df = pd.merge(df, label_df, on='Ind_ID')

In [14]:
def clean_and_encode_data(sorted_df: pd.DataFrame,
                        numerical_cols: List[str] = ['Annual_income', 'Birthday_count'], 
                          categorical_cols: List[str] = ['GENDER', 'Car_Owner', 'Propert_Owner', 'Type_Income', 
                                                         'EDUCATION', 'Marital_status', 'Housing_type', 'Type_Occupation'], 
                          occupation_col: str = 'Type_Occupation') -> pd.DataFrame:
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
    
    sorted_df.dropna(subset=[occupation_col], inplace=True)
    
    sorted_df['GENDER'] = sorted_df['GENDER'].fillna(sorted_df['GENDER'].mode()[0])
    
    sorted_df = pd.get_dummies(sorted_df, columns=categorical_cols, drop_first=True)

    sorted_df = sorted_df.replace([np.inf, -np.inf], np.nan).fillna(0)

    sorted_df = sorted_df.astype(int)
    scaler = StandardScaler()
    sorted_df[numerical_cols] = scaler.fit_transform(sorted_df[numerical_cols])
    return sorted_df
sorted_df = clean_and_encode_data(sorted_df)
sorted_df

,Ind_ID,CHILDREN,Annual_income,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Family_Members,...,Type_Occupation_Laborers,Type_Occupation_Low-skill Laborers,Type_Occupation_Managers,Type_Occupation_Medicine staff,Type_Occupation_Private service staff,Type_Occupation_Realty agents,Type_Occupation_Sales staff,Type_Occupation_Secretaries,Type_Occupation_Security staff,Type_Occupation_Waiters/barmen staff
8,5010864,1,2.047160,-0.909858,-678,1,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0
9,5010868,1,2.047160,-0.909858,-678,1,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0
10,5010869,1,2.047160,-0.909858,-678,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
11,5018498,0,-0.878721,-1.110597,-1002,1,1,1,0,2,...,0,0,0,0,0,0,0,0,0,0
12,5018501,0,-1.610191,-1.110597,-1002,1,1,1,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,5118268,1,1.315690,0.867338,-3536,1,0,1,0,3,...,0,0,0,0,0,0,0,0,0,0
1543,5028645,0,-1.610191,0.696052,-2182,1,0,0,0,2,...,0,0,1,0,0,0,0,0,0,0
1544,5023655,0,0.218485,1.142482,-1209,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1545,5115992,2,-0.147250,0.381638,-2477,1,0,0,0,4,...,0,0,1,0,0,0,0,0,0,0


# 3.
I normalized the numerical columns and tweaked the filling null values in addition to the data cleaning that was already done.

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [16]:
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,
                 hidden1=20,hidden2=20,
                 out_features=2):
        super().__init__() 
        """
        super is a computed indirect reference
        which means that it isolates changes and
        makes sure the children in the layers of
        multiple inheritance are calling 
        the right parents
        """
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)

    def forward(self, x):
        #apply activation function
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [17]:
torch.manual_seed(42)

# create an instance of the model
ann = ANN_Model()

In [18]:
def my_neural_network():
    X = sorted_df.drop('label', axis=1).values
    y = sorted_df['label'].values

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,
                                                        random_state=42,
                                                        stratify=y)
    
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)

    y_test = torch.LongTensor(y_test)
    y_train = torch.LongTensor(y_train)

    loss_function = nn.CrossEntropyLoss()


    optimizer = torch.optim.Adadelta(ann.parameters(), lr = 0.01)
    final_loss = []
    n_epochs = 100
    for epoch in range(n_epochs):
        y_pred = ann.forward(X_train)
        loss = loss_function(y_pred, y_train)
        final_loss.append(loss)

    optimizer.zero_grad() #zeros the gradient before running backward propagation
    loss.backward() # working to minimize the loss function using backward propagation
    optimizer.step() # perform one optimization step each function
    y_pred = []

    with torch.no_grad(): # this will decrease memory consumption
        for i, data in enumerate(X_test):
            prediction = ann(data)
            y_pred.append(prediction.argmax()) 
    
    y_pred = torch.tensor(y_pred).numpy()
    y_test = y_test.numpy()
    test_accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    return test_accuracy, conf_matrix, class_report
    


In [19]:
test_accuracy, conf_matrix, class_report = my_neural_network()
print(test_accuracy)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (742x45 and 8x20)